# Lab session 7: Outlier Detection

## Introduction

The purpose of this lab session is to provide you with an opportunity to gain experience in **outlier detection**, covered in lecture 8, using common Python libraries.

- This lab is the first part of a **two-lab assignment** that covers lectures 7 and 8, which is due on **Sunday, 24 July 2022, 11:59 PM**.
- The assignment will account for 10% of your overall grade. Questions in this lab sheet will contribute 5% of your overall grade.
- <font color = 'maroon'>The **assessed questions** are in the last section of this notebook (Assignment 3, part 2 of 2).</font> 

This session starts with a tutorial that uses examples to introduce you to the practical knowledge that you will need for the assignment. We highly recommend that you read the following tutorials if you need a gentler introduction to the libraries that we use:
- [Numpy quickstart tutorial](https://numpy.org/devdocs/user/quickstart.html)
- [Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html)
- [Matplotlib](https://matplotlib.org/tutorials/introductory/pyplot.html)
- [Scikit-learn](https://scikit-learn.org/stable/tutorial/basic/tutorial.html)

## Submission
- For this two-part assignment you are asked to submit:
    - A **report** with the answers to the assessed questions. The report should be in **PDF format** (so **NOT** *doc, docx, notebook* etc). It should contain your name, student number, assignment number (for instance, Assignment 1), module, and marked with question numbers. 
    - **Completed notebooks**. That is, the two notebooks for the assignment with the code you employed to answer the questions, at the end. That code should be standalone. That is, if someone opens your notebooks, they should be able to scroll down directly to your answers and run the corresponding cells. That means you may have to repeat some import statements, for example.
- No other means of submission other than submitting your assignment through the appropriate QM+ link are acceptable at any time. Submissions sent via email will **not** be considered.
- Please name your report as follows: Assignment3-StudentName-StudentNumber.pdf
- You may include other files that you think are necessary, but make sure you reference them clearly in the report.

## Important notes about the assignment: 
- **PLAGIARISM** <ins>is an irreversible non-negotiable failure in the course</ins> (if in doubt of what constitutes plagiarism, ask!). 
- The total assessed coursework is worth 40% of your final grade.
- There will be 9 lab sessions and 4 assignments.
- One assignment will cover 2 consecutive lab sessions and will be worth 10 marks (percentages of your final grade).
- The submission cut-off date will be 7 days after the deadline and penalties will be applied for late submissions in accordance with the School policy on late submissions.
- Cases of **Extenuating Circumstances (ECs)** have to go through the proper procedure of the School in due time. Only cases approved by the School in due time can be considered.
- For some of the below assignment questions, you can choose whether to answer them using pen-and-paper or using code. However, please note that questions which explicitly require loading CSV files or using a pre-loaded dataset must be addressed using code. A few assignment questions throughout the module might be easier to complete by hand, although could also be addressed using code. In both cases, either responding to questions manually or using code, please **do make sure to show your work** - i.e., how you derived the result, by showing the code that was used to generate the result that addresses the question and/or by writing down your reasoning or math derivations.
- Any mathematics included in the report must be written using Tex typesetting, or a system of comparable quality. A photograph of your handwritten derivations will not be accepted.

## 1. Outlier detection using parametric methods 

This approach assumes that the majority of the data instances are governed by some probability distribution, e.g. a Gaussian distribution. Outliers can then be detected by seeking observations that do not fit the overall distribution of the data.

In this example, our goal is to detect anomalous changes in the daily closing prices of various stocks. The input data **stocks.csv** contains the historical closing prices of stocks for 3 large corporations (Microsoft, Ford Motor Company, and Bank of America). 

In [ ]:
import pandas as pd

# Load CSV file, set the 'Date' values as the index of each row, and display the first rows of the dataframe
stocks = pd.read_csv('stocks.csv', header='infer') 
stocks.index = stocks['Date']
stocks = stocks.drop(['Date'],axis=1)
stocks.head()

We can compute the percentage of change in the daily closing price of each stock as follows:
\begin{equation}
\Delta(t) = 100 \times \frac{x_t - x_{t-1}}{x_{t-1}} 
\end{equation}

where $x_t$ denotes the price of a stock on day $t$ and $x_{t-1}$ denotes the price on its previous day, $t-1$.

In [ ]:
import numpy as np

N,d = stocks.shape
# Compute delta, which denotes the percentage of changes in the daily closing price of each stock
delta = pd.DataFrame(100*np.divide(stocks.iloc[1:,:].values-stocks.iloc[:N-1,:].values, stocks.iloc[:N-1,:].values),
                    columns=stocks.columns, index=stocks.iloc[1:].index)
delta.head()

We can now plot the daily percentage changes as a 3-dimensional scatter plot:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(8,5)).gca(projection='3d')
fig.scatter(delta.MSFT,delta.F,delta.BAC)
fig.set_xlabel('Microsoft')
fig.set_ylabel('Ford')
fig.set_zlabel('Bank of America')
plt.show()

Let's assume that the data follows a multivariate (i.e. multidimensional) Gaussian distribution. Such a probability distribution can be characterised by two statistics: the mean and covariance matrix of the 3-dimensional data. 

We can then compute the mean and covariance matrix of the 3-dimensional 'delta' data (which represent the percentage of changes in the daily closing price of each stock). Then, to determine the anomalous trading days we can compute the squared **Mahalanobis distance** between the percentage of price change on each day against the mean percentage of price change:
\begin{equation}
\textrm{MDist}(x,\bar{x}) = (x - \bar{x})^T S^{-1}(x - \bar{x})
\end{equation}
where $\bar{x}$ denotes the mean vector, and $S$ denotes the covariance matrix of the data.

See Section 12.3 in the "Data Mining: Concepts and Techniques" book for more information on the Mahalanobis distance. As a first step, we can define a function for the Mahalanobis distance:

In [ ]:
from numpy.linalg import inv

def mahalanobis(x=None, data=None):
    """Compute the Mahalanobis Distance between each row of x and the data  
    x    : vector or matrix of data with, say, p columns.
    data : ndarray of the distribution from which Mahalanobis distance of each observation of x is to be computed.
    """
    x_mu = x - data.mean()
    cov = np.cov(data.values.T)
    inv_covmat = np.linalg.inv(cov)
    left = np.matmul(x_mu, inv_covmat)
    mahal = np.dot(left, x_mu.T)
    return mahal.diagonal()

Then, we can call the created function for the Mahalanobis distance on the 'delta' dataframe containing the daily percentage changes for each stock:

In [ ]:
# Compute Mahalanobis distance for delta dataset
mahal = mahalanobis(x=delta, data=delta[['MSFT', 'F', 'BAC']])

# Assign an outlier score for the data based on the computed Mahalanobis distance
outlier_score = mahal

# Display 3D scatterplot with datapoints having a different color according to their outlier score
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111, projection='3d')
p = ax.scatter(delta.MSFT,delta.F,delta.BAC,c=outlier_score,cmap='jet')
ax.set_xlabel('Microsoft')
ax.set_ylabel('Ford')
ax.set_zlabel('Bank of America')
fig.colorbar(p)
plt.show()

The top outliers are shown as the dark red and orange points in the above scatterplot. We can examine the dates associated with the top-5 highest outlier scores as follows:

In [ ]:
outlier = pd.DataFrame(outlier_score, index=delta.index, columns=['Outlier score'])
result = pd.concat((delta,outlier), axis=1)
result.nlargest(5,'Outlier score')

We see for example that the top outlier was detected for 13th October 2008. This was a period of economic instability due to the 2008 global financial crisis (https://en.wikipedia.org/wiki/Global_financial_crisis_in_October_2008).

We can subsequently inspect stocks around the top-2 outlier dates for each company, and see to which compani(es) these outliers are attributed to.

In [ ]:
fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(15,6))

ts = delta[445:452]
ts.plot.line(ax=ax1)
ax1.set_xticks(range(7))
ax1.set_xticklabels(ts.index)
ax1.set_ylabel('Percent Change')

ts = delta[477:484]
ts.plot.line(ax=ax2)
ax2.set_xticks(range(7))
ax2.set_xticklabels(ts.index)
ax2.set_ylabel('Percent Change')

## 2. Outlier detection using proximity-based approaches

This is a model-free outlier detection approach, as it does not require constructing an explicit model of the normal class to determine the outlier score of data instances. The example code shown below employs the k-nearest neighbor approach to calculate the outlier score. Specifically, a normal instance is expected to have a small distance to its k-th nearest neighbour whereas an outlier is likely to have a large distance to its k-th nearest neighbour. In the example below, we apply the distance-based approach with k=4 to identify the anomalous trading days from the stock market data described in the previous section.

For more information on the NearestNeighbors() function please see the scikit learn documentation: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
from scipy.spatial import distance

# Implement a k-nearest neighbour approach using k=4 neighbours
k = 4
nbrs = NearestNeighbors(n_neighbors=k, metric=distance.euclidean).fit(delta.values)
distances, indices = nbrs.kneighbors(delta.values)

# The outlier score is set as the distance between the point and its k-th nearest neighbour
outlier_score = distances[:,k-1]

# Plot 3D scatterplot of outlier scores
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111, projection='3d')
p = ax.scatter(delta.MSFT,delta.F,delta.BAC,c=outlier_score,cmap='jet')
ax.set_xlabel('Microsoft')
ax.set_ylabel('Ford')
ax.set_zlabel('Bank of America')
fig.colorbar(p)
plt.show()

The results are slightly different to those shown in Section 1, due to the difference distance used (Euclidean distance vs Mahalanobis distance) and the proximity criterion to detect the outliers. 

We can examine the dates associated with the top-5 highest outlier scores as follows. 

In [ ]:
outlier = pd.DataFrame(outlier_score, index=delta.index, columns=['Outlier score'])
result = pd.concat((delta,outlier), axis=1)
result.nlargest(5,'Outlier score')

Finally, similarly to what was carried out in Section 1, we can inspect stocks around those outlier dates for each company. The below figure inspects the delta values for each company around the date of the 3rd detected outlier, on 7th October 2008, which represents a key date for the 2008 financial recession, with large drops in stock values. Two companies seem to be primarily responsible for this outlier.

In [ ]:
fig = plt.figure(figsize=(10,4))

ax = fig.add_subplot(111)
ts = delta[440:447]
ts.plot.line(ax=ax)
ax.set_xticks(range(7))
ax.set_xticklabels(ts.index)
ax.set_ylabel('Percent Change')

## 3. Outlier detection using classification-based methods

The **support vector machine (SVM)** algorithm, originally developed for binary classification, can also be used for one-class classification. This means we can use this method for outlier detection, by training a classifier for the normal class only.

When modeling one class, the algorithm captures the density of the majority class and classifies examples on the extremes of the density function as outliers. This modification of SVM is referred to as **One-Class SVM**.

In this section, we will work with a different dataset of house prices available at: https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv . This dataset has 13 input variables that describe the properties suburbs, and requires the prediction of the median value of houses in the suburb in thousands of dollars. Information and metadata about the dataset can be found at: https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.names . Please spend some time to inspect the dataset and its metadata.

As a first step, we load the dataset and split it into input and output (target) elements:

In [ ]:
from pandas import read_csv

# Loading the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
df = read_csv(url, header=None)

# Extracting the values from the dataframe
data = df.values

# Split dataset into input and output elements
X, y = data[:, :-1], data[:, -1]

# Summarize the shape of the dataset
print(X.shape, y.shape)

Using the OneClassSVM() function in scikit-learn, we can initialise and train an one-class SVM classifier on the input data. Please study the [OneClassSVM documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html) for information on input arguments. 

We can then print the estimated labels, which are -1 for outliers and 1 for inliers (i.e. data points that are considered normal).

In [ ]:
from sklearn.svm import OneClassSVM

ee = OneClassSVM(nu=0.01,gamma="scale")
yhat = ee.fit_predict(X) # Perform fit on input data and returns labels for that input data.

print(yhat) # Print labels: -1 for outliers and 1 for inliers.

Having trained the one-class SVM, we can then select all rows from the dataset that are **not** outliers:

In [ ]:
# Select all rows that are not outliers
mask = yhat != -1
X_filtered, y_filtered = X[mask, :], y[mask]

# Summarize the shape of the updated dataset
print(X_filtered.shape, y_filtered.shape)

## 4. Understanding the Mahalanobis distance

Let us try to understand the Mahalanobis distance a little bit better. To this end, we will use a normally distributed, synthetic, two-dimensional data set.

In [ ]:
X = np.loadtxt("mahalanobis.csv")
plt.scatter(X[:,0], X[:,1], s=2)
plt.ylim(-3,3)
plt.xlim(-3,3)

The plot shows that the attributes are correlated. Furthermore, the variance is much larger along one direction.

Computing Euclidean distances between these data points could be misleading in some cases, as we will illustrate in the following plot.

In [ ]:
plt.scatter(X[:,0], X[:,1], s=2)
Z = np.array([[0, .25], [0.1, -.25], [-2.25, -1], [2.25, 1]])
plt.scatter(Z[0,0], Z[0,1], s=50, color="red")
plt.scatter(Z[1,0], Z[1,1], s=50, color="red")
plt.scatter(Z[2,0], Z[2,1], s=50, color="green")
plt.scatter(Z[3,0], Z[3,1], s=50, color="green")
plt.ylim(-3,3)
plt.xlim(-3,3)

The Euclidean distance between the green points is be much larger than the distance between the red points, but both pairs represent points on opposite extremes of the distribution, and could therefore be considered equally different.

In [ ]:
def euclidean_distance(x, y):
    return np.linalg.norm(x-y)

In [ ]:
euclidean_distance(Z[0,:], Z[1,:]), euclidean_distance(Z[2,:], Z[3,:])

We will use the Mahalanobis distance instead, and try to understand how it overcomes this problem.

First, we compute the covariance matrix of the generated data set.

In [ ]:
S = np.cov(X.T)

We now compute the Mahalanobis distance between the example data points from before. We see that the distances are now closer, as we expected.

In [ ]:
Si = np.linalg.inv(S)
x,y = Z[0,:],Z[1,:]
print((x-y).dot(Si).dot(((x-y)).T))
x,y = Z[2,:],Z[3,:]
print((x-y).dot(Si).dot(((x-y)).T))

To visualize how this happened, we will apply explicitly the transformation that the Mahalanobis distance applies implicitly, and plot the data again.

In [ ]:
d,V = np.linalg.eigh(Si)
D = np.diag(d)
W = V.dot(np.sqrt(D))
Xh = X.dot(W)
plt.scatter(Xh[:,0], Xh[:,1], s=2)

Zh = Z.dot(W)
plt.scatter(Zh[0,0], Zh[0,1], s=50, color="red")
plt.scatter(Zh[1,0], Zh[1,1], s=50, color="red")
plt.scatter(Zh[2,0], Zh[2,1], s=50, color="green")
plt.scatter(Zh[3,0], Zh[3,1], s=50, color="green")

plt.ylim(-5,5)
plt.xlim(-5,5)

As shown by this plot, the Mahalanobis distance implicitly transforms the data so that the attributes are uncorrelated and have the same variance. 

## Assignment 3, part 2 of 2

Please **show your work** - i.e., show and explain your code/math, and write your reasoning.


1. **[1 mark]** The monthly rainfall in the London borough of Tower Hamlets in 2019 had the following amount of precipitation (measured in mm, values from January-December 2018): {22.93, 20.69, 25.75, 23.84, 25.34, 3.25, 23.55, 28.28, 23.72, 22.42, 26.83, 23.82}. Assuming that the data is based on a normal distribution, identify outlier values in the above dataset using the maximum likelihood method.

1. **[1 mark]** Using the house prices dataset from Section 3 of this lab notebook, use PCA to obtain the first 2 principal components (remember that PCA should only be applied on the input attributes, and not the target; remember also to normalise using z-scores for better results). Then, perform outlier detection on the pre-processed dataset using the k-nearest neighbours approach using k=2. Display a scatterplot of the two principal components, where each object is colour-coded according to the computed outlier score.

3. **[1 mark]** Consider the *absenteeism.csv* data set.

In [ ]:
df = pd.read_csv("absenteeism.csv", header="infer", delimiter=";")

a. The "ID" column is the employee identifier. Consider the number of absences for each employee and find outliers among those values. Use Grubb's test with $\alpha=0.1$ to find said outliers.
 
Use this website for reference if you have doubts: https://www.itl.nist.gov/div898/handbook/eda/section3/eda35h1.htm

Note: the code below can be used to compute the squared upper critical value at a significance of $\alpha/(2N)$ with $N-2$ degrees of freedom.

In [ ]:
from scipy.stats import t
N = 10
alpha=0.05
t2 = t.interval( 1-alpha/N, N-2)[1]**2
t2

b. The attribute "Reason for absence" is categorical. The label "26" corresponds to "Unjustified absence". Again, consider the number of absences for each employee. However, now you must find contextual outliers among cases of unjustified absences, again using Grubb's test with $\alpha=0.1$.

-------------------------

For the next exercise we will consider the following implementation of the Nested-Loops algorithm to detect outliers.

It takes as arguments a numerical dataset $X$ (in numpy array form), a radius $r$ and a threshold $\pi$. It returns a list of outliers (points having less than $\pi\times n$ neighbours within a distance of at most $r$).

The implementation assumes access to a distance function, defined in the cell below as well.

In [ ]:
import numpy as np

distance = euclidean_distance

def detect_outliers(X, r, pi):
    N = X.shape[0]
    outliers = np.ones(N)
    for i in range(N):
        x = X[i,:]
        count = 0
        for j in range(N):
            if i == j: # No need to compute the distance of x to itself
                continue
            y = X[j,:]
            dist = distance(x,y)
            if dist <= r:
                count += 1
            if count >= pi*N:
                outliers[i] = 0
                break    
    return np.where(outliers)[0]

We can test it on the *normal.npy* dataset.

In [ ]:
X = np.load("normal.npy")
r=8
pi=0.1
outliers = detect_outliers(X, r, pi)
outliers

This algorithm saves some time by breaking loops early, but it still very slow. We can test its performance as follows:

In [ ]:
%timeit detect_outliers(X, r, pi)

4. **[1 mark]** Propose a method to improve the running time of the above algorithm using the triangle inequality. Recall the triangle inequality: for any metric distance function $d$ and any three points $x,y,z$, we have $d(x,z) \leq d(x,y) + d(y,z)$. For this exercise, it is enough to describe the idea and explain why it would work. It is not necessary to implement it.

<details>
<summary>Hint (click to see):</summary>
  
Start by computing the distances between one point (e.g. the mean of the data set) and all the rest.
</details>

5. **[1 mark]** Implement an algorithm based on your proposal. Check that it returns the same list of outliers as the original nested loop algorithm (make sure you use the same values of $r$ and $\pi$). Test its speed as above.